In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from IPython.display import display
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from transformers import TextStreamer

In [ ]:
# Hyperparameters
SECONDS_BEFORE = 0.5  # number of seconds before the collision for image frame.

In [ ]:
PROJECT_DIR = "nexcar-challenge"
DATA_DIR = "data"

train_csv_path = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/train.csv"
train_videos_folder = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/train/"

df = pd.read_csv(train_csv_path)
# df = df[df['target']==1]
df.head()

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data, videos_folder, transform=None):
        self.data = data
        self.videos_folder = videos_folder
        self.transform = transform  # Any image transformations (e.g., augmentations)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        vid_path = os.path.join(
            self.videos_folder, f"{str(int(row['id'])).zfill(5)}.mp4"
        )

        time = (
            0.0
            if np.isnan(row["time_of_event"])
            else row["time_of_event"] - SECONDS_BEFORE
        )
        image = self.get_frame(vid_path, time)

        label = row["target"]

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label), row['id']  # Convert label to tensor

    def get_frame(self, video_path, time_sec):
        cap = cv2.VideoCapture(video_path)
        cap.set(cv2.CAP_PROP_POS_MSEC, time_sec * 1000)

        success, frame = cap.read()
        cap.release()

        if not success:
            raise ValueError(f"Failed to read frame at {time_sec} seconds.")

        # Convert BGR to RGB
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return Image.fromarray(frame)


transform = transforms.Compose(
    [
        transforms.Resize((480, 854)),  # Resize to a standard size
    ]
)

dataset = ImageDataset(df, train_videos_folder, transform=transform)
dataloader = DataLoader(dataset, shuffle=True)

In [ ]:
image, label, _ = dataset[0]
display(image)
print(label)

In [ ]:
instruction = """Examine the provided dashcam image and analyze the positions and trajectories of all visible vehicles and obstacles. Based solely on the visual cues in the image, determine if there is an imminent risk of collision. Answer with only "Yes" or "No"."""


def convert_to_conversation(image, label):
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": instruction},
                {"type": "image", "image": image},
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": "Yes" if label else "No"}],
        },
    ]
    return {"messages": conversation}


converted_dataset = [
    convert_to_conversation(image, label) for image, label, _ in tqdm(dataset)
]

In [ ]:
converted_dataset[0]

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from unsloth import FastVisionModel  # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit",  # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit",  # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",
    "unsloth/Pixtral-12B-2409-bnb-4bit",  # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",  # Pixtral base model
    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",  # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",
    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",  # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
]  # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    # "unsloth/Llama-3.2-11B-Vision-Instruct",
    fourbit_models[0],
    load_in_4bit=True,  # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for long context
)

In [ ]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers=True,  # False if not finetuning vision layers
    finetune_language_layers=True,  # False if not finetuning language layers
    finetune_attention_modules=True,  # False if not finetuning attention layers
    finetune_mlp_modules=True,  # False if not finetuning MLP layers
    r=16,  # The larger, the higher the accuracy, but might overfit
    lora_alpha=16,  # Recommended alpha == r at least
    lora_dropout=0,
    bias="none",
    random_state=3407,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

image, label, _ = dataset[0]
conversation = convert_to_conversation(image, label)

input_text = tokenizer.apply_chat_template(
    conversation["messages"], add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

In [ ]:
from unsloth import is_bf16_supported
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model)  # Enable for training!

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=UnslothVisionDataCollator(model, tokenizer),  # Must use!
    train_dataset=converted_dataset,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        # num_train_epochs = 1, # Set this instead of max_steps for full training runs
        learning_rate=2e-4,
        fp16=not is_bf16_supported(),
        bf16=is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # For Weights and Biases
        # You MUST put the below items for vision finetuning:
        remove_unused_columns=False,
        dataset_text_field="",
        dataset_kwargs={"skip_prepare_dataset": True},
        dataset_num_proc=4,
        max_seq_length=2048,
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
FastVisionModel.for_inference(model)  # Enable for inference!

image, label, _ = dataset[0]
conversation = convert_to_conversation(image, label)

input_text = tokenizer.apply_chat_template(
    conversation["messages"], add_generation_prompt=True
)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors="pt",
).to("cuda")


text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    min_p=0.1
)

In [ ]:
model.save_pretrained(
    f"/content/drive/MyDrive/{PROJECT_DIR}/pretrained/finetuned_llama/"
)
tokenizer.save_pretrained(
    f"/content/drive/MyDrive/{PROJECT_DIR}/pretrained/finetuned_llama/"
)

In [ ]:
correct = 0
total = len(dataset)

for i, (image, target, _) in enumerate(tqdm(dataset)):
    conversation = convert_to_conversation(image, label)
    input_text = tokenizer.apply_chat_template(
        conversation["messages"], add_generation_prompt=True
    )
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    # Generate output and decode text
    output_tokens = model.generate(
        **inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1
    )
    output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True).lower()

    # Evaluate the output:
    if (target == 1 and "yes" in output_text.lower()) or (
        target != 1 and "no" in output_text.lower()
    ):
        correct += 1

accuracy = correct / total
print("Accuracy:", accuracy)

In [ ]:
test_csv_path = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/test.csv"
test_videos_folder = f"/content/drive/MyDrive/{PROJECT_DIR}/{DATA_DIR}/test/"
test_set = ImageDataset(df, train_videos_folder, transform=transform)

In [ ]:
results = []

for image, target, id in test_set:
    conversation = convert_to_conversation(image, label)
    input_text = tokenizer.apply_chat_template(
        conversation["messages"], add_generation_prompt=True
    )

    # Tokenize the image and text inputs and move them to CUDA.
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    output_tokens = model.generate(
        **inputs, max_new_tokens=128, use_cache=True, temperature=1.5, min_p=0.1
    )

    target = 1 if "yes" in output_text.lower() else 0
    results.append({"id": id, "target": target})

df = pd.DataFrame(results)
df.to_csv("test_results.csv", index=False)
print("Test set results saved to 'test_results.csv'")